In [55]:
import pandas as pd

In [56]:
cluster4 = pd.read_csv('cluster4.csv', index_col=0)
#cluster4.drop(cluster4.loc[1], inplace=True)
cluster4.reset_index(drop = True, inplace=True)
cluster4.head(15)

,Cluster Label,Zip Code,City,Community,Estimated Median Income,Longitude,Latitude,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size,Factor
0,4,90011,Los Angeles,Los Angeles (Southeast Los Angeles),40940,-118.25868,34.007063,103892,26.2,52794,51098,22168,4.67,4253338480
1,4,90019,Los Angeles,"Los Angeles (Arlington Heights, Country Club P...",56389,-118.34015,34.048411,64458,35.8,31442,33016,23344,2.70,3634722162
2,4,90022,Los Angeles,East Los Angeles,46051,-118.15581,34.023662,67179,29.8,33216,33963,17023,3.94,3093660129
3,4,90026,Los Angeles,"Los Angeles (Echo Park, Silver Lake)",65269,-118.26596,34.078510,67869,34.0,34515,33354,24956,2.68,4429741761
4,4,90044,Los Angeles,"Athens, Los Angeles (South Los Angeles)",35981,-118.29158,33.953814,89779,28.6,43128,46651,25144,3.55,3230338199
5,4,90201,Bell,"Bell, Bell Gardens, Cudahy",44046,-118.17385,33.972814,101279,27.8,50658,50621,24104,4.16,4460934834
6,4,90250,Hawthorne,Hawthorne (Holly Park),56304,-118.35092,33.914614,93193,31.9,45113,48080,31087,2.98,5247138672
7,4,90255,Huntington Park,"Huntington Park, Walnut Park",44962,-118.21773,33.978404,75066,29.1,37525,37541,18419,4.06,3375117492
8,4,90262,Lynwood,Lynwood,52313,-118.20186,33.924642,69745,27.8,33919,35826,14669,4.57,3648570185
9,4,90280,South Gate,South Gate,52321,-118.19490,33.944264,94396,29.4,46321,48075,23278,4.05,4938893116


In [57]:
import foursquare
import requests
CLIENT_ID = foursquare.CLIENT_ID
CLIENT_SECRET = foursquare.CLIENT_SECRET
VERSION = '20210501' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [58]:
record = cluster4.iloc[10]

In [105]:
import itertools
offset = 0
venues_list=[]

for _ in itertools.repeat(None, 350):
    url = 'https://api.foursquare' \
          '.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},' \
          '{}&radius={}&limit={}&offset={}&categoryId={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        record['Latitude'],
        record['Longitude'],
        3500,
        LIMIT,
        offset,
        '4d4b7105d754a06374d81259')

    # make the GET request
   # try:
    try:
        results = requests.get(url).json()["response"]['venues']
        for v in results:
            try:
                city = v['location']['city']
            except:
                city = 'N. A.'

            venues_list.append((
                        record['Zip Code'],
                        record['Community'],
                        record['Latitude'],
                        record['Longitude'],
                        v['name'],
                        # v['venue']['location']['lat'],
                        # v['venue']['location']['lng'],
                        v['categories'][0]['name'],
                        city ))
    except:
        print('done!')
   # except:
   #     print('done')
    offset += 50

#nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues = pd.DataFrame(venues_list, columns=['Zip Code', 'Community',
              'Zip Code Latitude', 'Zip Code Longitude', 'Venue',
              # 'Venue Latitude',
              # 'Venue Longitude',
              'Venue Category',
              'City'])
nearby_venues.count()

done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!


Zip Code              16950
Community             16950
Zip Code Latitude     16950
Zip Code Longitude    16950
Venue                 16950
Venue Category        16950
City                  16950
dtype: int64

In [106]:
import numpy as np
np.sum(nearby_venues['City'] == 'Norwalk')
#nearby_venues[nearby_venues['City']== 'Norwalk'].count()

8475

In [107]:
nearby_venues.head()

,Zip Code,Community,Zip Code Latitude,Zip Code Longitude,Venue,Venue Category,City
0,90650,Norwalk,33.906914,-118.08339,Starbucks,Coffee Shop,Norwalk
1,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk
2,90650,Norwalk,33.906914,-118.08339,El Pollo Loco,Fast Food Restaurant,Artesia
3,90650,Norwalk,33.906914,-118.08339,Taco Bell,Fast Food Restaurant,Norwalk
4,90650,Norwalk,33.906914,-118.08339,Taco Nazo,Mexican Restaurant,Bellflower


In [97]:
# nearby_venues.to_csv(path_or_buf='nearby_venues.csv')

In [110]:
#ppds = nearby_venues.drop(nearby_venues[nearby_venues['City'] !=
#                             'Norwalk'].index)

ppds = nearby_venues[nearby_venues['City']=='Norwalk']
ppds.head(10)

,Zip Code,Community,Zip Code Latitude,Zip Code Longitude,Venue,Venue Category,City
0,90650,Norwalk,33.906914,-118.08339,Starbucks,Coffee Shop,Norwalk
1,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk
3,90650,Norwalk,33.906914,-118.08339,Taco Bell,Fast Food Restaurant,Norwalk
8,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk
12,90650,Norwalk,33.906914,-118.08339,Starbucks,Coffee Shop,Norwalk
13,90650,Norwalk,33.906914,-118.08339,Yoshinoya,Donburi Restaurant,Norwalk
18,90650,Norwalk,33.906914,-118.08339,In-N-Out Burger,Fast Food Restaurant,Norwalk
19,90650,Norwalk,33.906914,-118.08339,Stater Bros. Markets,Grocery Store,Norwalk
20,90650,Norwalk,33.906914,-118.08339,7-Eleven,Convenience Store,Norwalk
22,90650,Norwalk,33.906914,-118.08339,Taco Bell,Fast Food Restaurant,Norwalk


In [111]:
ppds['City'].count()

8475

In [116]:
#ppds.to_csv(path_or_buf='nearby_venues_3500.csv')

In [112]:
df = ppds[['Venue Category', 'Zip Code']]
df_grouped = df.groupby('Venue Category').agg(['mean', 'count'])
df_sorted = df_grouped.sort_values(by=[('Zip Code','count')], ascending=False)
#df_grouped.sort_values(by='count')
df_sorted

Zip Code      
                         mean count
Venue Category                     
Fast Food Restaurant    90650  3390
Coffee Shop             90650  1017
Grocery Store           90650   678
Breakfast Spot          90650   339
Burger Joint            90650   339
Convenience Store       90650   339
Donburi Restaurant      90650   339
Donut Shop              90650   339
Korean Restaurant       90650   339
Mexican Restaurant      90650   339
Steakhouse              90650   339
Supermarket             90650   339
Taco Place              90650   339

In [113]:
ppds.head(20)

,Zip Code,Community,Zip Code Latitude,Zip Code Longitude,Venue,Venue Category,City
0,90650,Norwalk,33.906914,-118.08339,Starbucks,Coffee Shop,Norwalk
1,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk
3,90650,Norwalk,33.906914,-118.08339,Taco Bell,Fast Food Restaurant,Norwalk
8,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk
12,90650,Norwalk,33.906914,-118.08339,Starbucks,Coffee Shop,Norwalk
13,90650,Norwalk,33.906914,-118.08339,Yoshinoya,Donburi Restaurant,Norwalk
18,90650,Norwalk,33.906914,-118.08339,In-N-Out Burger,Fast Food Restaurant,Norwalk
19,90650,Norwalk,33.906914,-118.08339,Stater Bros. Markets,Grocery Store,Norwalk
20,90650,Norwalk,33.906914,-118.08339,7-Eleven,Convenience Store,Norwalk
22,90650,Norwalk,33.906914,-118.08339,Taco Bell,Fast Food Restaurant,Norwalk


In [114]:
df = ppds[['Venue Category']]
df_count = df.groupby('Venue Category').size().to_frame('Count').reset_index()
df_sorted = df_count.sort_values(by=[('Count')], ascending=False)
df_sorted

,Venue Category,Count
6,Fast Food Restaurant,3390
2,Coffee Shop,1017
7,Grocery Store,678
0,Breakfast Spot,339
1,Burger Joint,339
3,Convenience Store,339
4,Donburi Restaurant,339
5,Donut Shop,339
8,Korean Restaurant,339
9,Mexican Restaurant,339


In [115]:
import plotly.express as px
fig = px.bar(df_sorted, x="Venue Category", y="Count", title='Distribution of Food Venues in Norwalk')
fig.show()